<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-13 14:14:23
-------------------
qualified stocks: 93
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  2.51 L
Current:  1.33 C
-------------------
Today PnL: 69.72 K (0.53%)
Current PnL: -21.73 L (-14.78%)
CY Booked + Current PnL: -13.46 L (-9.15%)
-------------------
Total profit:  2.60 L
Total loss:  -24.32 L
-------------------
Total Booked + Current PnL: 13.52 L (11.27%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.21%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.14 C
Est FTT PnL: 81.20 L (60.98%)
Deployed:  1.20 C
Current:  1.33 C
CAGR/XIRR %: 6.66%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.04,112344.0,8032.0,8617.0,5.26,7.70,7.67,15.96,11.0,0.93,0.86,25.90,X5K,ATH,METALS
68,SBIN,760.30,863.00,-12.30,M-LC,3.85,217804.0,16324.0,10890.0,0.16,8.10,5.00,13.51,61.0,1.50,1.67,21.72,XY25,NTT,BANKS
4,APOLLOHOSP,7098.95,8285.00,-14.86,H-LC,5.62,170038.0,13861.0,12226.0,6.81,8.88,7.19,16.71,19.0,1.13,1.30,27.70,X40N,BTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,10.70,H-MC,13.93,158015.0,31815.0,14364.0,-1.31,25.21,9.09,36.59,111.0,2.21,1.21,40.88,XY24,NTT,HEALTHCARE
83,TTKPRESTIG,769.29,770.00,93.38,M-SC,10.53,83277.0,-17500.0,17596.0,0.43,-17.37,21.13,0.09,243.0,-0.99,0.64,9.41,OX40N,NTT,DURABLES


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DABUR,505.20,735.00,-9.78,H-MC,3.74,196228.0,-800.0,90422.0,0.05,-0.41,46.08,45.49,103.0,-0.01,1.50,12.58,XY24,BTT,FMCG
90,VOLTAS,1278.28,1918.49,-8.21,H-MC,7.66,192450.0,708.0,95320.0,3.05,0.37,49.53,50.08,101.0,0.01,1.47,7.19,XY25,ATH,AC
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.05,100626.0,122.0,30520.0,1.79,0.12,30.33,30.49,80.0,0.00,0.77,32.57,MH,ATH,METALS
92,WIPRO,243.46,420.00,-14.70,M-LC,7.15,150214.0,-731.0,110182.0,0.23,-0.48,73.35,72.51,54.0,-0.01,1.15,5.51,XR,NTT,IT


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-20.12,H-LC,2.30,213035.0,8535.0,32871.0,2.20,4.17,15.43,20.25,10.0,0.26,1.63,13.27,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-13.06,H-LC,2.82,169483.0,-31903.0,119519.0,0.57,-15.84,70.52,43.51,27.0,-0.27,1.30,17.98,X40,BTT,PAINTS
57,NESTLEIND,2268.60,2755.0,-16.62,H-LC,3.61,128384.0,-137042.0,193950.0,0.57,-51.63,151.07,21.44,8.0,-0.71,0.98,3.60,XY25,NTT,FMCG
53,LTIM,5564.16,7230.2,-7.23,H-LC,3.63,183852.0,-16458.0,76427.0,0.80,-8.22,41.57,29.94,17.0,-0.22,1.41,27.01,X200,ATH,IT
4,APOLLOHOSP,7098.95,8285.0,-14.86,H-LC,5.62,170038.0,13861.0,12226.0,6.81,8.88,7.19,16.71,19.0,1.13,1.30,27.70,X40N,BTT,HEALTHCARE


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,7098.95,8285.00,-14.86,H-LC,5.62,170038.0,13861.0,12226.0,6.81,8.88,7.19,16.71,19.0,1.13,1.30,27.70,X40N,BTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.90,72749.0,-40800.0,80853.0,5.99,-35.93,111.14,35.27,268.0,-0.50,0.56,78.05,XR,NTT,HOTELS
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.04,112344.0,8032.0,8617.0,5.26,7.70,7.67,15.96,11.0,0.93,0.86,25.90,X5K,ATH,METALS
88,VALIANTORG,512.64,838.00,-313.97,H-SC,17.23,90987.0,-42299.0,126890.0,4.42,-31.74,139.46,63.47,143.0,-0.33,0.70,52.52,XR,NTT,CHEMICALS
75,SURYODAY,189.78,240.00,42.54,H-SC,4.62,100922.0,-49004.0,88680.0,4.15,-32.69,87.87,26.46,137.0,-0.55,0.77,29.29,XR,NTT,BANKS


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BATAINDIA,1550.24,2096.0,-38.72,M-SC,6.56,91698.0,-36972.0,82271.0,-2.73,-28.73,89.72,35.20,209.0,-0.45,0.70,0.00,X40,NTT,FOOTWEAR
30,GREENPANEL,375.16,537.0,193.93,M-SC,5.75,131150.0,-51928.0,130901.0,-2.52,-28.36,99.81,43.14,230.0,-0.40,1.00,23.36,XY24,NTT,MISC
59,QUESS,296.83,986.0,-40.50,M-SC,42.04,58845.0,-6161.0,157087.0,-2.43,-9.48,266.95,232.18,198.0,-0.04,0.45,0.56,XY24,NTT,MISC
29,EASEMYTRIP,18.12,26.4,-10.04,M-SC,24.18,86749.0,-90066.0,170861.0,-2.31,-50.94,196.96,45.70,197.0,-0.53,0.66,0.91,XY24,NTT,TRAVEL
72,SIS,477.00,528.0,2151.81,H-SC,6.82,60580.0,-20510.0,29181.0,-2.09,-25.29,48.17,10.69,155.0,-0.70,0.46,22.77,OX40N,NTT,MISC


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TTKPRESTIG,769.29,770.0,93.38,M-SC,10.53,83277.0,-17500.0,17596.0,0.43,-17.37,21.13,0.09,243.0,-0.99,0.64,9.41,OX40N,NTT,DURABLES
84,UJJIVANSFB,52.77,60.0,83.20,H-SC,20.55,115938.0,-26541.0,46062.0,-0.53,-18.63,39.73,13.70,163.0,-0.58,0.89,36.84,OX40N,NTT,BANKS
49,KANSAINER,299.63,340.0,-68.89,H-SC,2.50,214335.0,-55332.0,91671.0,-0.09,-20.52,42.77,13.47,142.0,-0.60,1.64,8.41,XY24,NTT,PAINTS
20,CERA,7989.07,9475.0,-20.53,H-SC,3.09,113544.0,-30259.0,57010.0,-0.76,-21.04,50.21,18.60,148.0,-0.53,0.87,25.30,OX40N,NTT,CERAMICS
43,INDIGOPNTS,1407.73,1408.0,109.26,M-SC,3.11,137169.0,-37390.0,37420.0,0.65,-21.42,27.28,0.02,221.0,-1.00,1.05,18.95,OX40N,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
80,TCS,3794.03,4998.00,-28.36,H-LC,15.83,237276.0,-58658.0,152568.0,0.22,-19.82,64.30,31.73,2.0,-0.38,1.82,1.30,X40,BTT,IT
45,INFY,1461.46,2275.00,-21.75,H-LC,10.62,255755.0,-5846.0,151458.0,0.33,-2.23,59.22,55.67,4.0,-0.04,1.96,3.73,X40,BTT,IT
89,VBL,492.64,671.64,-9.05,H-LC,10.43,276209.0,11169.0,85128.0,-0.17,4.21,30.82,36.33,5.0,0.13,2.11,18.07,X40N,ATH,FMCG
1,ABB,5551.76,7934.00,-41.62,H-LC,10.16,198276.0,-18243.0,111154.0,0.25,-8.43,56.06,42.91,7.0,-0.16,1.52,3.84,AR,NTT,ELECTRICAL
57,NESTLEIND,2268.60,2755.00,-16.62,H-LC,3.61,128384.0,-137042.0,193950.0,0.57,-51.63,151.07,21.44,8.0,-0.71,0.98,3.60,XY25,NTT,FMCG


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,10.7,H-MC,13.93,158015.0,31815.0,14364.0,-1.31,25.21,9.09,36.59,111.0,2.21,1.21,40.88,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.00,10.70,H-MC,13.93,158015.0,31815.0,14364.0,-1.31,25.21,9.09,36.59,111.0,2.21,1.21,40.88,XY24,NTT,HEALTHCARE
50,KPIGREEN,497.21,731.35,9.40,H-SC,12.67,130208.0,4911.0,54088.0,0.36,3.92,41.54,47.09,141.0,0.09,1.00,62.97,MH,ATH,POWER
15,BLUESTARCO,1646.70,2326.38,2.59,H-MC,7.14,177210.0,12540.0,55431.0,1.52,7.62,31.28,41.28,91.0,0.23,1.36,16.21,X40N,ATH,AC
19,CAMS,3643.00,5211.76,-5.61,H-SC,3.47,107195.0,5191.0,38730.0,1.58,5.09,36.13,43.06,123.0,0.13,0.82,23.08,X40N,ATH,MISC
90,VOLTAS,1278.28,1918.49,-8.21,H-MC,7.66,192450.0,708.0,95320.0,3.05,0.37,49.53,50.08,101.0,0.01,1.47,7.19,XY25,ATH,AC


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-9.65,H-MC,7.42,167223.0,16310.0,40986.0,2.85,10.81,24.51,37.97,92.0,0.40,1.28,38.85,AR,ATH,AUTO
42,INDIAMART,2327.09,4850.92,-51.65,H-SC,12.19,134885.0,11549.0,122219.0,0.94,9.36,90.61,108.45,119.0,0.09,1.03,33.81,AR,ATH,MISC
50,KPIGREEN,497.21,731.35,9.40,H-SC,12.67,130208.0,4911.0,54088.0,0.36,3.92,41.54,47.09,141.0,0.09,1.00,62.97,MH,ATH,POWER
91,WHIRLPOOL,1219.98,2270.00,-44.58,M-SC,4.01,93608.0,2110.0,76646.0,0.40,2.31,81.88,86.07,219.0,0.03,0.72,35.41,XR,NTT,DURABLES
56,NATIONALUM,189.63,247.44,-49.46,H-MC,2.05,100626.0,122.0,30520.0,1.79,0.12,30.33,30.49,80.0,0.00,0.77,32.57,MH,ATH,METALS


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.90,72749.0,-40800.0,80853.0,5.99,-35.93,111.14,35.27,268.0,-0.50,0.56,78.05,XR,NTT,HOTELS
3,ALKYLAMINE,2347.98,4546.37,-2.99,H-SC,16.26,90326.0,-10637.0,105167.0,1.32,-10.54,116.43,93.63,149.0,-0.10,0.69,34.54,SR,ATH,CHEMICALS
54,MASFIN,326.60,399.50,-18.70,H-SC,14.97,92250.0,-5730.0,27601.0,0.82,-5.85,29.92,22.32,152.0,-0.21,0.71,33.79,XR,ATH,FINANCE
36,HINDALCO,651.95,756.01,-18.25,H-LC,6.04,112344.0,8032.0,8617.0,5.26,7.70,7.67,15.96,11.0,0.93,0.86,25.90,X5K,ATH,METALS
84,UJJIVANSFB,52.77,60.00,83.20,H-SC,20.55,115938.0,-26541.0,46062.0,-0.53,-18.63,39.73,13.70,163.0,-0.58,0.89,36.84,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
70,SHALBY,261.39,327.00,965.39,M-SC,7.10,144989.0,-36154.0,81614.0,1.57,-19.96,56.29,25.10,232.0,-0.44,1.11,16.19,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-9.65,H-MC,7.42,167223.0,16310.0,40986.0,2.85,10.81,24.51,37.97,92.0,0.40,1.28,38.85,AR,ATH,AUTO
51,LALPATHLAB,2838.53,3545.00,-25.84,H-SC,9.57,221428.0,28408.0,19641.0,0.01,14.72,8.87,24.89,134.0,1.45,1.69,41.50,X40N,NTT,HEALTHCARE
23,COFFEEDAY,59.14,80.00,-51.71,L-SC,21.90,72749.0,-40800.0,80853.0,5.99,-35.93,111.14,35.27,268.0,-0.50,0.56,78.05,XR,NTT,HOTELS
89,VBL,492.64,671.64,-9.05,H-LC,10.43,276209.0,11169.0,85128.0,-0.17,4.21,30.82,36.33,5.0,0.13,2.11,18.07,X40N,ATH,FMCG


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.43
1,25,41.15
2,50,70.90


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    39.40
LC    34.30
MC    26.28
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.91
X40N     15.48
X40      13.31
XY25     10.68
XR        9.05
AR        8.42
OX40N     7.52
X5K       2.29
MH        1.77
X200      1.41
SR        1.14
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    25.94
H-SC    24.54
H-MC    23.33
M-SC    13.37
M-LC     7.27
M-MC     2.61
L-SC     1.49
L-LC     1.09
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.23,-10.87,46.51
IT,11.96,-19.51,79.19
FINANCE,8.30,-20.17,64.63
BANKS,7.66,-14.04,62.47
MISC,5.84,-20.15,89.01
PAINTS,5.66,-18.17,41.46
HEALTHCARE,5.31,5.46,18.41
ELECTRICAL,4.71,-14.17,63.62
AUTO,4.48,-15.27,65.41


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2856571.0,23
XR,1113056.0,13
AR,1109253.0,10
X40,832687.0,10
XY25,653323.0,8
X40N,584342.0,12
OX40N,476163.0,10
SR,194855.0,2
X5K,138881.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2810606.0,28
M-SC,1497788.0,17
H-LC,1348508.0,18
H-MC,1330377.0,18
M-LC,427770.0,5
M-MC,336608.0,2
L-SC,265926.0,3
L-MC,58512.0,1
L-LC,44071.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      937739.0      6
M-SC       XY24      872846.0      7
H-SC       AR        674514.0      5
           XR        524318.0      6
H-LC       X40       518597.0      5
H-MC       XY24      494517.0      5
M-MC       XY24      336608.0      2
H-LC       XY25      256469.0      3
H-MC       X40       231819.0      4
H-LC       AR        231608.0      2
H-MC       XY25      219934.0      2
H-SC       X40N      219355.0      4
H-LC       X40N      218363.0      5
H-SC       OX40N     205737.0      4
           SR        194855.0      2
M-LC       XY24      176434.0      2
L-SC       XR        176333.0      2
M-SC       AR        162145.0      2
H-MC       X40N      146624.0      3
M-SC       OX40N     132568.0      4
M-LC       X5K       130264.0      1
M-SC       XR        125999.0      2
           XY25      121959.0      1
H-MC       XR        117712.0      1
M-LC       XR        110182.0      1
L-SC       OX40N      89593.0      1
M-SC       X40        82271.0      1
H-LC       X200       76427.0      1
L-MC       XR         58512.0      1
H-SC       MH         54088.0      1
H-MC       OX40N      48265.0      1
L-LC       XY25       44071.0      1
H-MC       AR         40986.0      1
H-LC       XY24       38427.0      1
H-MC       MH         30520.0      1
M-LC       XY25       10890.0      1
H-LC       X5K         8617.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 35.0 seconds
